This Notebook was  excecuted  in kaggle for the Resource shortage 

In [56]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import cv2 
import pydicom 
import os 
import tensorflow as tf 
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, UpSampling2D, Concatenate, Input, GlobalAveragePooling2D, Dense,Dropout, Flatten , concatenate
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import Precision, Recall, AUC

# 1. Data preparation 

# 1.1 overall dataset creation

In [3]:
df  = pd.read_csv("/kaggle/input/rsna-completedatsetcsv/complite_train_dataset.csv")

In [4]:
df.drop(columns = ['image_file_path'], inplace = True )

In [5]:
df.head()

,study_id,series_id,series_description,instance_number,condition,level,x,y,severity
0,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild
1,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild
2,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild
3,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Normal/Mild
4,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Normal/Mild


In [6]:
df['series_description'].unique()

array(['Sagittal T2/STIR', 'Sagittal T1', 'Axial T2'], dtype=object)

In [7]:
fileter_segatiel_t2 = df[df['series_description'] == "Sagittal T2/STIR" ]
filter_sagatial_Sagittal_T1  = df[df['series_description'] == "Sagittal T1"]
filter_sagatial_Axial_T2 = df[df['series_description'] == "Axial T2"]

In [8]:
print(f"sagatial_T2{fileter_segatiel_t2['condition'].unique()}\n sagatial_T1:{filter_sagatial_Sagittal_T1['condition'].unique()}\nAxial_T2:{filter_sagatial_Axial_T2['condition'].unique()}")

sagatial_T2['Spinal Canal Stenosis']
 sagatial_T1:['Right Neural Foraminal Narrowing' 'Left Neural Foraminal Narrowing'
 'Spinal Canal Stenosis']
Axial_T2:['Left Subarticular Stenosis' 'Right Subarticular Stenosis']


In [9]:
fileter_sagatail_T1 = filter_sagatial_Sagittal_T1[filter_sagatial_Sagittal_T1['condition'] == 'Spinal Canal Stenosis']

In [10]:
fileter_sagatail_T1

,study_id,series_id,series_description,instance_number,condition,level,x,y,severity
41259,3637444890,3951475160,Sagittal T1,8,Spinal Canal Stenosis,L1/L2,214.852201,109.703066,Normal/Mild
41260,3637444890,3951475160,Sagittal T1,8,Spinal Canal Stenosis,L2/L3,204.388986,150.468734,Normal/Mild
41261,3637444890,3951475160,Sagittal T1,8,Spinal Canal Stenosis,L3/L4,201.250021,194.466270,Moderate
41262,3637444890,3951475160,Sagittal T1,8,Spinal Canal Stenosis,L4/L5,205.435307,233.520513,Severe
41263,3637444890,3951475160,Sagittal T1,8,Spinal Canal Stenosis,L5/S1,216.944844,265.956480,Normal/Mild


In [11]:
filter_sagatial_Sagittal_T1.head()

,study_id,series_id,series_description,instance_number,condition,level,x,y,severity
5,4003253,1054713880,Sagittal T1,4,Right Neural Foraminal Narrowing,L4/L5,187.961759,251.839388,Moderate
6,4003253,1054713880,Sagittal T1,4,Right Neural Foraminal Narrowing,L5/S1,198.240918,285.613767,Normal/Mild
7,4003253,1054713880,Sagittal T1,5,Right Neural Foraminal Narrowing,L3/L4,187.227533,210.722753,Moderate
8,4003253,1054713880,Sagittal T1,6,Right Neural Foraminal Narrowing,L1/L2,194.569790,127.755258,Normal/Mild
9,4003253,1054713880,Sagittal T1,6,Right Neural Foraminal Narrowing,L2/L3,191.632887,165.934990,Normal/Mild


In [12]:
fileter_segatiel_t2.head()

,study_id,series_id,series_description,instance_number,condition,level,x,y,severity
0,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild
1,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild
2,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild
3,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Normal/Mild
4,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Normal/Mild


In [13]:
filter_sagatial_Axial_T2.head() 

,study_id,series_id,series_description,instance_number,condition,level,x,y,severity
15,4003253,2448190387,Axial T2,3,Left Subarticular Stenosis,L1/L2,179.126448,161.235521,Normal/Mild
16,4003253,2448190387,Axial T2,4,Right Subarticular Stenosis,L1/L2,145.288771,158.624642,Normal/Mild
17,4003253,2448190387,Axial T2,11,Left Subarticular Stenosis,L2/L3,180.979730,158.764479,Normal/Mild
18,4003253,2448190387,Axial T2,11,Right Subarticular Stenosis,L2/L3,145.900042,157.096466,Normal/Mild
19,4003253,2448190387,Axial T2,19,Left Subarticular Stenosis,L3/L4,176.037645,157.528958,Normal/Mild


In [14]:
print(f"sagatial T1:{len(filter_sagatial_Sagittal_T1)} \n sagatail T2:{len(fileter_segatiel_t2)} \n Axial:{len(filter_sagatial_Axial_T2)}")

sagatial T1:19724 
 sagatail T2:9748 
 Axial:19220


In [15]:


def list_all_images(image_folder):
    all_images = []
    for root, dirs, files in os.walk(image_folder):
        for file in files:
            if file.endswith('.dcm'):
                full_path = os.path.join(root, file)
                study_id = os.path.basename(os.path.dirname(os.path.dirname(full_path)))
                series_id = os.path.basename(os.path.dirname(full_path))
                instance_number = os.path.splitext(file)[0]
                all_images.append((study_id, series_id, instance_number, full_path))
    return pd.DataFrame(all_images, columns=['study_id', 'series_id', 'instance_number', 'image_path'])

image_folder = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'  # Update with the correct path
all_images_df = list_all_images(image_folder)
all_images_df.head()


,study_id,series_id,instance_number,image_path
0,1737682527,2291122880,12,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
1,1737682527,2291122880,18,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
2,1737682527,2291122880,9,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
3,1737682527,2291122880,14,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
4,1737682527,2291122880,11,/kaggle/input/rsna-2024-lumbar-spine-degenerat...


In [16]:
len(all_images_df)

147218

In [17]:
# Load the condition CSV file
condition_csv_path = '/kaggle/input/rsna-completedatsetcsv/complite_train_dataset.csv'
condition_df = pd.read_csv(condition_csv_path)

# Create a unique identifier for each image
condition_df['image_id'] = condition_df['study_id'].astype(str) + '_' + condition_df['series_id'].astype(str) + '_' + condition_df['instance_number'].astype(str)
condition_df.head()


,study_id,series_id,series_description,instance_number,condition,level,x,y,image_file_path,severity,image_id
0,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,4003253_702807833_8.dcm,Normal/Mild,4003253_702807833_8
1,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,4003253_702807833_8.dcm,Normal/Mild,4003253_702807833_8
2,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,4003253_702807833_8.dcm,Normal/Mild,4003253_702807833_8
3,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,4003253_702807833_8.dcm,Normal/Mild,4003253_702807833_8
4,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,4003253_702807833_8.dcm,Normal/Mild,4003253_702807833_8


In [18]:
# Create a unique identifier for each image in all_images_df
all_images_df['image_id'] = all_images_df['study_id'].astype(str) + '_' + all_images_df['series_id'].astype(str) + '_' + all_images_df['instance_number'].astype(str)

# Find non-condition images
non_condition_df = all_images_df[~all_images_df['image_id'].isin(condition_df['image_id'])]
non_condition_df.head()


,study_id,series_id,instance_number,image_path,image_id
0,1737682527,2291122880,12,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_12
1,1737682527,2291122880,18,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_18
2,1737682527,2291122880,9,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_9
3,1737682527,2291122880,14,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_14
5,1737682527,2291122880,10,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_10


In [19]:
len(non_condition_df)

122672

In [20]:
# Add a label column for non-condition images, set to 'No Condition'
non_condition_df['condition'] = 'No Condition'
non_condition_df['level'] = 'None'
non_condition_df['x'] = None
non_condition_df['y'] = None

# Save non-condition images to a csv file 
non_condition_csv_path = 'non_condition_images.csv'
non_condition_df.to_csv(non_condition_csv_path, index=False)

print(f"Non-condition CSV file created at: {non_condition_csv_path}")


/tmp/ipykernel_34/2028163953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_condition_df['condition'] = 'No Condition'
/tmp/ipykernel_34/2028163953.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_condition_df['level'] = 'None'
/tmp/ipykernel_34/2028163953.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

Non-condition CSV file created at: non_condition_images.csv


In [21]:
non_condition_csv  = pd.read_csv("/kaggle/working/non_condition_images.csv")
condition_csv  = pd.read_csv("/kaggle/input/rsna-completedatsetcsv/complite_train_dataset.csv")

In [22]:
non_condition_csv['label'] = 'No condition'

In [23]:
len(non_condition_csv)

122672

In [24]:
non_condition_csv.head()

,study_id,series_id,instance_number,image_path,image_id,condition,level,x,y,label
0,1737682527,2291122880,12,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_12,No Condition,NaN,NaN,NaN,No condition
1,1737682527,2291122880,18,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_18,No Condition,NaN,NaN,NaN,No condition
2,1737682527,2291122880,9,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_9,No Condition,NaN,NaN,NaN,No condition
3,1737682527,2291122880,14,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_14,No Condition,NaN,NaN,NaN,No condition
4,1737682527,2291122880,10,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,1737682527_2291122880_10,No Condition,NaN,NaN,NaN,No condition


In [25]:
condition_csv.drop(columns  = ['severity'] , inplace  = True)

In [26]:
condition_csv['label'] = condition_csv['condition'] + '_'+ condition_csv['level']

In [27]:
condition_csv.head()

,study_id,series_id,series_description,instance_number,condition,level,x,y,image_file_path,label
0,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,4003253_702807833_8.dcm,Spinal Canal Stenosis_L1/L2
1,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,4003253_702807833_8.dcm,Spinal Canal Stenosis_L2/L3
2,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,4003253_702807833_8.dcm,Spinal Canal Stenosis_L3/L4
3,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,4003253_702807833_8.dcm,Spinal Canal Stenosis_L4/L5
4,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,4003253_702807833_8.dcm,Spinal Canal Stenosis_L5/S1


In [28]:
condition_csv.drop(columns = ['image_file_path'] ,  inplace = True )

In [29]:
print(f"Condition images :{len(condition_csv)}\nnon condition images :{len(non_condition_csv)}")

Condition images :48692
non condition images :122672


In [30]:
# Adjust the number to match your desired balance  Downsample the non-condition dataset 
non_condition_csv_downsampled = non_condition_csv.sample(n=48692, random_state=42)



In [31]:
print(f"Condition images :{len(condition_csv)}\nnon condition images  after downsample:{len(non_condition_csv_downsampled)}")

Condition images :48692
non condition images  after downsample:48692


In [32]:
train_des  = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv")

In [33]:
train_des

,study_id,series_id,series_description
0,4003253,702807833,Sagittal T2/STIR
1,4003253,1054713880,Sagittal T1
2,4003253,2448190387,Axial T2
3,4646740,3201256954,Axial T2
4,4646740,3486248476,Sagittal T1
...,...,...,...
6289,4287160193,1507070277,Sagittal T2/STIR
6290,4287160193,1820446240,Axial T2
6291,4290709089,3274612423,Sagittal T2/STIR
6292,4290709089,3390218084,Axial T2


In [34]:
non_condition_csv_downsampled.drop(columns = ['image_path','image_id'],inplace = True)

In [35]:
condition_images = condition_csv.reindex(sorted(condition_csv.columns), axis=1)
non_condition_images = non_condition_csv_downsampled.reindex(sorted(non_condition_csv_downsampled.columns), axis=1)
merged_dataset = pd.concat([condition_images, non_condition_images], ignore_index=True)

In [36]:
merged_dataset

,condition,instance_number,label,level,series_description,series_id,study_id,x,y
0,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L1/L2,L1/L2,Sagittal T2/STIR,702807833,4003253,322.831858,227.964602
1,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L2/L3,L2/L3,Sagittal T2/STIR,702807833,4003253,320.571429,295.714286
2,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L3/L4,L3/L4,Sagittal T2/STIR,702807833,4003253,323.030303,371.818182
3,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L4/L5,L4/L5,Sagittal T2/STIR,702807833,4003253,335.292035,427.327434
4,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L5/S1,L5/S1,Sagittal T2/STIR,702807833,4003253,353.415929,483.964602
...,...,...,...,...,...,...,...,...,...
97379,No Condition,14,No condition,NaN,NaN,12078898,1423031356,NaN,NaN
97380,No Condition,21,No condition,NaN,NaN,3637464851,1271819130,NaN,NaN
97381,No Condition,13,No condition,NaN,NaN,647735603,1681401548,NaN,NaN
97382,No Condition,15,No condition,NaN,NaN,2762843442,2348053690,NaN,NaN


In [37]:
overall_dataset = merged_dataset
series_descriptions = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')
merged_dataset = pd.merge(overall_dataset, series_descriptions[['series_id', 'series_description']], on='series_id', how='left')
merged_dataset.to_csv('merged_dataset_with_descriptions.csv', index=False)

print(f"Number of records in the merged dataset: {len(merged_dataset)}")


Number of records in the merged dataset: 97384


In [38]:
merged_dataset.drop(columns= ['series_description_x' ], inplace = True)

In [39]:
merged_dataset

,condition,instance_number,label,level,series_id,study_id,x,y,series_description_y
0,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L1/L2,L1/L2,702807833,4003253,322.831858,227.964602,Sagittal T2/STIR
1,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L2/L3,L2/L3,702807833,4003253,320.571429,295.714286,Sagittal T2/STIR
2,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L3/L4,L3/L4,702807833,4003253,323.030303,371.818182,Sagittal T2/STIR
3,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L4/L5,L4/L5,702807833,4003253,335.292035,427.327434,Sagittal T2/STIR
4,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L5/S1,L5/S1,702807833,4003253,353.415929,483.964602,Sagittal T2/STIR
...,...,...,...,...,...,...,...,...,...
97379,No Condition,14,No condition,NaN,12078898,1423031356,NaN,NaN,Axial T2
97380,No Condition,21,No condition,NaN,3637464851,1271819130,NaN,NaN,Sagittal T2/STIR
97381,No Condition,13,No condition,NaN,647735603,1681401548,NaN,NaN,Sagittal T2/STIR
97382,No Condition,15,No condition,NaN,2762843442,2348053690,NaN,NaN,Sagittal T1


# 1.2 Encoding 

In [40]:
# Encode the description and the level
series_description_encoder = {desc: idx for idx, desc in enumerate(merged_dataset['series_description_y'].unique())}
merged_dataset['series_description_encode'] = merged_dataset['series_description_y'].map(series_description_encoder)

# Encode levels
label_encoder = {level: idx for idx, level in enumerate(merged_dataset['label'].unique())}
merged_dataset['label_encoder'] = merged_dataset['label'].map(label_encoder)

In [41]:
print(series_description_encoder)
print(label_encoder)

{'Sagittal T2/STIR': 0, 'Sagittal T1': 1, 'Axial T2': 2}
{'Spinal Canal Stenosis_L1/L2': 0, 'Spinal Canal Stenosis_L2/L3': 1, 'Spinal Canal Stenosis_L3/L4': 2, 'Spinal Canal Stenosis_L4/L5': 3, 'Spinal Canal Stenosis_L5/S1': 4, 'Right Neural Foraminal Narrowing_L4/L5': 5, 'Right Neural Foraminal Narrowing_L5/S1': 6, 'Right Neural Foraminal Narrowing_L3/L4': 7, 'Right Neural Foraminal Narrowing_L1/L2': 8, 'Right Neural Foraminal Narrowing_L2/L3': 9, 'Left Neural Foraminal Narrowing_L1/L2': 10, 'Left Neural Foraminal Narrowing_L4/L5': 11, 'Left Neural Foraminal Narrowing_L5/S1': 12, 'Left Neural Foraminal Narrowing_L2/L3': 13, 'Left Neural Foraminal Narrowing_L3/L4': 14, 'Left Subarticular Stenosis_L1/L2': 15, 'Right Subarticular Stenosis_L1/L2': 16, 'Left Subarticular Stenosis_L2/L3': 17, 'Right Subarticular Stenosis_L2/L3': 18, 'Left Subarticular Stenosis_L3/L4': 19, 'Right Subarticular Stenosis_L3/L4': 20, 'Left Subarticular Stenosis_L4/L5': 21, 'Right Subarticular Stenosis_L4/L5': 22

In [42]:
reverse_series_description_encoder = {idx: cond for cond, idx in series_description_encoder.items()}
reverse_label_encoder = {idx: cond for cond, idx in label_encoder.items()}

In [43]:
reverse_label_encoder[23] , reverse_series_description_encoder[0]

('Left Subarticular Stenosis_L5/S1', 'Sagittal T2/STIR')

In [44]:
merged_dataset

,condition,instance_number,label,level,series_id,study_id,x,y,series_description_y,series_description_encode,label_encoder
0,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L1/L2,L1/L2,702807833,4003253,322.831858,227.964602,Sagittal T2/STIR,0,0
1,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L2/L3,L2/L3,702807833,4003253,320.571429,295.714286,Sagittal T2/STIR,0,1
2,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L3/L4,L3/L4,702807833,4003253,323.030303,371.818182,Sagittal T2/STIR,0,2
3,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L4/L5,L4/L5,702807833,4003253,335.292035,427.327434,Sagittal T2/STIR,0,3
4,Spinal Canal Stenosis,8,Spinal Canal Stenosis_L5/S1,L5/S1,702807833,4003253,353.415929,483.964602,Sagittal T2/STIR,0,4
...,...,...,...,...,...,...,...,...,...,...,...
97379,No Condition,14,No condition,NaN,12078898,1423031356,NaN,NaN,Axial T2,2,25
97380,No Condition,21,No condition,NaN,3637464851,1271819130,NaN,NaN,Sagittal T2/STIR,0,25
97381,No Condition,13,No condition,NaN,647735603,1681401548,NaN,NaN,Sagittal T2/STIR,0,25
97382,No Condition,15,No condition,NaN,2762843442,2348053690,NaN,NaN,Sagittal T1,1,25


In [45]:
merged_dataset['series_description_encode'].unique()  , merged_dataset['label_encoder'].unique() 

(array([0, 1, 2]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25]))

In [46]:
final_df = merged_dataset[['study_id','series_id','instance_number','x','y','series_description_encode', 'label_encoder']]

In [47]:
final_df['series_description_encode'].unique()

array([0, 1, 2])

In [48]:
final_df['label_encoder'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25])

In [49]:

def get_file_path(row):
    base_path  = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
    # Convert the IDs to integers to remove the .0 suffix
    study_id = int(row['study_id'])
    series_id = int(row['series_id'])
    instance_number = int(row['instance_number'])
    file_path  = os.path.join(base_path, str(study_id), str(series_id), f"{instance_number}.dcm")
    return file_path

final_df['file_path'] = final_df.apply(lambda row: get_file_path(row), axis=1)


/tmp/ipykernel_34/2901677349.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['file_path'] = final_df.apply(lambda row: get_file_path(row), axis=1)


In [50]:
final_df.iloc[0]['file_path']

'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/4003253/702807833/8.dcm'

# 2.Data Process 

In [2]:
import os 
os.makedirs('preproces/medical_image_process.py')

# 2.1 Data preprocess Function 

In [ ]:
#Data preprocess 
# Mask Function
def generate_mask_from_coordinates(image_shape, x, y, mask_size=25):
    xmin = int(np.floor(x - mask_size / 2))
    xmax = int(np.ceil(x + mask_size / 2))
    ymin = int(np.floor(y - mask_size / 2))
    ymax = int(np.ceil(y + mask_size / 2))
    
    xmin = max(0, xmin)
    ymin = max(0, ymin)
    xmax = min(image_shape[1], xmax)
    ymax = min(image_shape[0], ymax)
    
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    mask[ymin:ymax, xmin:xmax] = 255
    
    return mask, (xmin, ymin, xmax, ymax)
#Coodinates function
def adjust_coordinates(x, y, original_shape, new_shape):
    original_height, original_width = original_shape[:2]
    new_height, new_width = new_shape[:2]
    x_new = x * new_width / original_width 
    y_new = y * new_height / original_height 
    return x_new, y_new
#Image preprocess FUnction 
def medical_preprocess(image_path, description, label, original_x=None, original_y=None, new_shape=(224, 224)):
    try:
        if isinstance(image_path, np.ndarray):
            image_path = image_path.item()
        if isinstance(image_path, bytes):
            image_path = image_path.decode()
            
        meta_data = pydicom.dcmread(image_path)
        pixel_array = meta_data.pixel_array
        
        image = cv2.resize(pixel_array, new_shape)
        image = image / np.max(image)
        image = np.stack((image,)*3, axis=-1)
        
        original_shape = pixel_array.shape
        
        if original_x is not None and original_y is not None:
            new_x, new_y = adjust_coordinates(original_x, original_y, original_shape, new_shape)
            mask, bbox = generate_mask_from_coordinates(image.shape, new_x, new_y)
        else:
            mask = np.zeros(new_shape[:2], dtype=np.uint8)
            
        return image, description, mask, label 
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return np.zeros(new_shape[:2], dtype=np.uint8), description, np.zeros(new_shape[:2], dtype=np.uint8), label


In [ ]:
# Visualize Function

def visualize(image, mask=None, label=None):
    if mask is not None:
         if len(mask.shape) == 4: 
                mask = np.squeeze(mask, axis=0) 
                mask = np.squeeze(mask, axis=-1) 
    plt.figure(figsize=(12, 6))

    # Plot original image
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')

    # Overlay mask and plot coordinates
    plt.subplot(1, 2, 2)
    plt.imshow(image, cmap='gray')
    if mask is not None:
        plt.imshow(mask, cmap='Reds', alpha=0.5)  
    if  x is not None and y is not None :
        plt.scatter(x, y, c='blue', s=50) 
    if label is not None:
        plt.title(f'Image with Mask and Coordinates: {label}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()


# 2.2 Custom DataGenerator

In [ ]:
#Custom DataGenrator
class CustomDataGenerator(Sequence):
    def __init__(self, dataframe, batch_size=32, shuffle=True, new_shape=(224, 224)):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.new_shape = new_shape
        self.indexes = np.arange(len(self.dataframe))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        start_index = index * self.batch_size
        end_index = (index + 1) * self.batch_size
        batch_indexes = self.indexes[start_index:end_index]

        batch_images, batch_descriptions, batch_labels, batch_masks = [], [], [], []

        for idx in batch_indexes:
            row = self.dataframe.iloc[idx]
            image_path = row['file_path']
            description = row['series_description_encode']
            label = row['label_encoder']
            x = row['x']
            y = row['y']

            if np.isnan(x) or np.isnan(y):
                x = None
                y = None

            image, description, mask, label = medical_preprocess(image_path, description, label, x, y, self.new_shape)

            batch_images.append(image)
            batch_descriptions.append(description)
            batch_labels.append(label)
            batch_masks.append(mask)

        batch_images = np.array(batch_images, dtype=np.float32)
        batch_descriptions = np.array(batch_descriptions, dtype=np.float32)
        batch_labels = np.array(batch_labels, dtype=np.int32)
        batch_masks = np.array(batch_masks, dtype=np.float32)

        inputs = (tf.convert_to_tensor(batch_images, dtype=tf.float32), tf.convert_to_tensor(batch_descriptions, dtype=tf.float32))
        targets = (tf.convert_to_tensor(batch_masks, dtype=tf.float32), tf.convert_to_tensor(batch_labels, dtype=tf.int32))

        return inputs, targets

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


# 3.Model 

# 3.1 Model creation 

In [ ]:
%%writefile model/segmentation_model.py
# Create the model function
from tensorflow.keras.regularizers import l2
def create_model(input_shape=(224, 224, 3), num_classes=26):
    # Define image input
    image_input = Input(shape=input_shape, name='image_input')
    
    # Define description input
    desc_input = Input(shape=(1,), dtype=tf.float32, name='desc_input')
    base_model = EfficientNetB0(include_top=False, input_tensor=image_input, weights=None)
    
    # Load the pretrained weights
    base_model.load_weights("/kaggle/input/pretrained-model/efficientnetb0_notop.h5")
    
    # Freeze the base model layers
    base_model.trainable = False


    # Encoder network
    conv1 = base_model.get_layer('block2a_expand_activation').output
    conv2 = base_model.get_layer('block3a_expand_activation').output
    conv3 = base_model.get_layer('block4a_expand_activation').output
    conv4 = base_model.get_layer('block6a_expand_activation').output
    conv5 = base_model.output

    # Decoder network
    up6 = UpSampling2D((2, 2))(conv5)
    up6 = Conv2D(512, (2, 2), padding='same',kernel_regularizer=l2(0.01))(up6)
    merge6 = Concatenate()([conv4, up6])

    up7 = UpSampling2D((2, 2))(merge6)
    up7 = Conv2D(256, (2, 2), padding='same',kernel_regularizer=l2(0.01))(up7)
    merge7 = Concatenate()([conv3, up7])

    up8 = UpSampling2D((2, 2))(merge7)
    up8 = Conv2D(128, (2, 2), padding='same',kernel_regularizer=l2(0.01))(up8)
    merge8 = Concatenate()([conv2, up8])

    up9 = UpSampling2D((2, 2))(merge8)
    up9 = Conv2D(64, (2, 2), padding='same',kernel_regularizer=l2(0.01))(up9)
    up9 = Dropout(0.5)(up9) 
    merge9 = Concatenate()([conv1, up9])

    up10 = UpSampling2D((2, 2))(merge9)
    mask_output = Conv2D(1, (1, 1), activation='sigmoid', name='mask_output')(up10)
    
    desc_flatten = Flatten()(desc_input)
#     desc_dense = Dense(128, activation='relu')(desc_flatten)
    desc_dense = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(desc_flatten)
    global_features = GlobalAveragePooling2D()(conv5)
    combined = Concatenate()([global_features, desc_dense])

    label_output = Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01) ,  name='label_output')(combined)
#     label_output = Dense(num_classes, activation='softmax', name='label_output')(combined)

    model = Model(inputs=[image_input, desc_input], outputs=[mask_output, label_output])
    
    return model


In [64]:
#intialize segmebt model 
segmentation_model = create_model(input_shape=(224, 224, 3), num_classes=26)

In [65]:
segmentation_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ image_input[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_2     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ normalization_2[… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_

 Total params: 8,245,022 (31.45 MB)

 Trainable params: 4,195,451 (16.00 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [59]:
# learning rate schedular 
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)


In [60]:
len(final_df)

97384

In [61]:
# train_size = int(0.8 * len(final_df))
# train_df = final_df.iloc[:train_size]
# val_df = final_df.iloc[train_size:val_size]
train_size  = 90000
val_size  = 7384
train_df, val_df = train_test_split(final_df, test_size=val_size, train_size=train_size)

train_generator = CustomDataGenerator(train_df, batch_size=32)
val_generator = CustomDataGenerator(val_df, batch_size=32)



In [67]:

segmentation_model.compile(optimizer='adam', 
              loss={'mask_output': 'binary_crossentropy', 'label_output': 'sparse_categorical_crossentropy'},
              metrics={'mask_output': 'accuracy', 'label_output': 'accuracy'})
early_stopping = EarlyStopping(monitor='val_loss', patience=20)
checkpoint = ModelCheckpoint('segmentation_model_checkpoint.keras', save_best_only=True)


print("Starting model training...")
history = segmentation_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[early_stopping, checkpoint , lr_scheduler],
    verbose=1
)
print("Model training completed.")




Starting model training...
Epoch 1/5
   1/2812 ━━━━━━━━━━━━━━━━━━━━ 29:15:16 37s/step - label_output_accuracy: 0.5000 - loss: -5.6354 - mask_output_accuracy: 0.0752

W0000 00:00:1721816614.760127     134 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2812/2812 ━━━━━━━━━━━━━━━━━━━━ 0s 634ms/step - label_output_accuracy: 0.5012 - loss: -8.3022 - mask_output_accuracy: 0.0911

W0000 00:00:1721818405.302065     134 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2812/2812 ━━━━━━━━━━━━━━━━━━━━ 1967s 686ms/step - label_output_accuracy: 0.5012 - loss: -8.3024 - mask_output_accuracy: 0.0911 - val_label_output_accuracy: 0.5041 - val_loss: -9.8444 - val_mask_output_accuracy: 0.0761 - learning_rate: 0.0010
Epoch 2/5
2812/2812 ━━━━━━━━━━━━━━━━━━━━ 1575s 559ms/step - label_output_accuracy: 0.4962 - loss: -11.9298 - mask_output_accuracy: 0.2751 - val_label_output_accuracy: 0.5041 - val_loss: -18.1850 - val_mask_output_accuracy: 0.6987 - learning_rate: 0.0010
Epoch 3/5
2812/2812 ━━━━━━━━━━━━━━━━━━━━ 1635s 580ms/step - label_output_accuracy: 0.4974 - loss: -16.7939 - mask_output_accuracy: 0.5586 - val_label_output_accuracy: 0.5048 - val_loss: 30.5846 - val_mask_output_accuracy: 0.9466 - learning_rate: 0.0010
Epoch 4/5
2812/2812 ━━━━━━━━━━━━━━━━━━━━ 1565s 555ms/step - label_output_accuracy: 0.5019 - loss: -12.5503 - mask_output_accuracy: 0.3657 - val_label_output_accuracy: 0.5045 - val_loss: -18.5656 - val_mask_output_accuracy: 0.6248 - learning_rate: 0.00

In [ ]:
segmentation_model.save("segmentation")

In [100]:
segmentation_model.evaluate(val_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 640ms/step - label_output_accuracy: 0.4375 - loss: -13.3954 - mask_output_accuracy: 0.0758


[-13.395435333251953, 0.4375, 0.07581314444541931]

In [13]:
def get_prediction(segmentation_model ,image_path, description, image_shape=(224, 224)):
    meta_data = pydicom.dcmread(image_path)
    pixel_array = meta_data.pixel_array
    image_data = cv2.resize(pixel_array, image_shape)
    image_data = image_data / np.max(image_data)
    image_data = np.stack((image_data,)*3, axis=-1)
    image_data = np.expand_dims(image_data, axis=0)
    description = np.expand_dims(np.array(description), axis=0)
    mask_data, label = segmentation_model.predict([image_data, description])
    label = np.argmax(label)
    label_encode  = reverse_label_encoder[label]
    
    return image_data , mask_data, label , label_encode 


In [102]:
get_prediction("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/2828203845/1.dcm" ,0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


(array([[[[0.00364964, 0.00364964, 0.00364964],
          [0.00273723, 0.00273723, 0.00273723],
          [0.00729927, 0.00729927, 0.00729927],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
 
         [[0.03193431, 0.03193431, 0.03193431],
          [0.0629562 , 0.0629562 , 0.0629562 ],
          [0.05656934, 0.05656934, 0.05656934],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
 
         [[0.05748175, 0.05748175, 0.05748175],
          [0.05656934, 0.05656934, 0.05656934],
          [0.0419708 , 0.0419708 , 0.0419708 ],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
 
         ...,
 
         [[0.04835766, 0.04835766, 0.04835766],
          [0.02737

In [103]:
# Visualize the cropped image and original image 
def visualize_cropped(original_image, cropped_image, mask, label):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(original_image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(cropped_image, cmap='gray')
    plt.imshow(mask, cmap='Reds', alpha=0.5) 
    plt.title('Cropped Image with Mask Overlay')
    plt.axis('off')
    plt.suptitle(f'Label: {label}', fontsize=16)
    plt.tight_layout()
    plt.show()


In [ ]:
# segment_image  = segment_image(image , mask)

#  DataGenerator For Severity Model 

In [ ]:
class CustomSeverityDataGenerator(Sequence):
    def __init__(self, dataframe, batch_size=32, shuffle=True, new_shape=(224, 224)):
        self.data = dataframe 
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.new_shape = new_shape
        self.indexes = np.arange(len(self.data))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        start_index = index * self.batch_size
        end_index = (index + 1) * self.batch_size
        batch_indexes = self.indexes[start_index:end_index]

        batch_descriptions, cropped_images, labels, conditions = [], [], [], []

        for idx in batch_indexes:
            row = self.data.iloc[idx]
            image_path = row['file_path']
            description = row['series_description_encode']
            x = row['x']
            y = row['y']
            condition = row['condition_encode']
            label = row['severity_encoder']
            
            if np.isnan(x) or np.isnan(y):
                x = None
                y = None


            image_data, mask = self.__medical_preprocess_cropped(image_path, x, y)

            cropped_image, batch_description, condition, label = self.__cropped_image(image_data, description, condition, mask, label)

            cropped_images.append(cropped_image)
            batch_descriptions.append(batch_description)
            labels.append(label)
            conditions.append(condition)

        cropped_images = np.array(cropped_images, dtype=np.float32)
        batch_descriptions = np.array(batch_descriptions, dtype=np.int32)
        labels = np.array(labels, dtype=np.int32)
        conditions = np.array(conditions, dtype=np.int32)

        inputs = (tf.convert_to_tensor(cropped_images, dtype=tf.float32), tf.convert_to_tensor(batch_descriptions, dtype=tf.float32), tf.convert_to_tensor(conditions, dtype=tf.int32))
        targets = (tf.convert_to_tensor(labels, dtype=tf.int32))

        return inputs, targets

    def __cropped_image(self, image_data, description,condition,  mask, label, new_shape=(224 , 224)):
        xmin, ymin, xmax, ymax = self.__get_bounding_box(mask)
        cropped_image = image_data[ymin:ymax + 1, xmin:xmax + 1]
        cropped_image = cv2.resize(cropped_image, new_shape)
        cropped_image = cropped_image / 255.0
        return cropped_image, description,condition , label

    def __get_bounding_box(self, mask):
        rows = np.any(mask, axis=1)
        cols = np.any(mask, axis=0)
        y_min, y_max = np.where(rows)[0][[0, -1]]
        x_min, x_max = np.where(cols)[0][[0, -1]]
        return x_min, y_min, x_max, y_max

    def __generate_mask_from_coordinates(self, image_shape, x, y, mask_size=25):
        xmin = int(np.floor(x - mask_size / 2))
        xmax = int(np.ceil(x + mask_size / 2))
        ymin = int(np.floor(y - mask_size / 2))
        ymax = int(np.ceil(y + mask_size / 2))

        xmin = max(0, xmin)
        ymin = max(0, ymin)
        xmax = min(image_shape[1], xmax)
        ymax = min(image_shape[0], ymax)

        mask = np.zeros(image_shape[:2], dtype=np.uint8)
        mask[ymin:ymax, xmin:xmax] = 255

        return mask, (xmin, ymin, xmax, ymax)

    def __adjust_coordinates(self, x, y, original_shape, new_shape):
        original_height, original_width = original_shape[:2]
        new_height, new_width = new_shape[:2]
        x_new = x * new_width / original_width
        y_new = y * new_height / original_height
        return x_new, y_new

    def __medical_preprocess_cropped(self, image_path, original_x=None, original_y=None, new_shape=(224, 224)):
        try:
            if isinstance(image_path, np.ndarray):
                image_path = image_path.item()
            if isinstance(image_path, bytes):
                image_path = image_path.decode()

            meta_data = pydicom.dcmread(image_path)
            pixel_array = meta_data.pixel_array

            image = cv2.resize(pixel_array, new_shape)
            image = image / np.max(image)
            image = np.stack((image,)*3, axis=-1)

            original_shape = pixel_array.shape

            if original_x is not None and original_y is not None:
                new_x, new_y = self.__adjust_coordinates(original_x, original_y, original_shape, new_shape)
                mask, bbox = self.__generate_mask_from_coordinates(image.shape, new_x, new_y)
            else:
                mask = np.zeros(new_shape[:2], dtype=np.uint8)

            return image, mask
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            return np.zeros(new_shape[:2], dtype=np.uint8), np.zeros(new_shape[:2], dtype=np.uint8)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


In [4]:
cropped_image_df = pd.read_csv("/kaggle/input/rsna-completedatsetcsv/complite_train_dataset.csv")

In [5]:
cropped_image_df = cropped_image_df.dropna(subset = ['severity'])

In [6]:
cropped_image_df.isna().sum()

study_id              0
series_id             0
series_description    0
instance_number       0
condition             0
level                 0
x                     0
y                     0
image_file_path       0
severity              0
dtype: int64

In [7]:

def get_file_path(row):
    base_path  = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
    # Convert the IDs to integers to remove the .0 suffix
    study_id = int(row['study_id'])
    series_id = int(row['series_id'])
    instance_number = int(row['instance_number'])
    file_path  = os.path.join(base_path, str(study_id), str(series_id), f"{instance_number}.dcm")
    return file_path

cropped_image_df['file_path'] = cropped_image_df.apply(lambda row: get_file_path(row), axis=1)


In [8]:
cropped_image_df['file_path'].iloc[1]

'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/4003253/702807833/8.dcm'

In [9]:
cropped_image_df.head(2)

,study_id,series_id,series_description,instance_number,condition,level,x,y,image_file_path,severity,file_path
0,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,4003253_702807833_8.dcm,Normal/Mild,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
1,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,4003253_702807833_8.dcm,Normal/Mild,/kaggle/input/rsna-2024-lumbar-spine-degenerat...


In [10]:
series_descriptions = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')

cropped_image_df = pd.merge(cropped_image_df, series_descriptions[['series_id', 'series_description']], on='series_id', how='left')


In [11]:
cropped_image_df['condition_leve'] = cropped_image_df['condition']+'_'+cropped_image_df['level']

In [12]:
cropped_image_df.head(1)

,study_id,series_id,series_description_x,instance_number,condition,level,x,y,image_file_path,severity,file_path,series_description_y,condition_leve
0,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,4003253_702807833_8.dcm,Normal/Mild,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Sagittal T2/STIR,Spinal Canal Stenosis_L1/L2


In [13]:
# Encode the description and the severity  


series_description_encoder = {desc: idx for idx, desc in enumerate(cropped_image_df['series_description_x'].unique())}
cropped_image_df['series_description_encode'] = cropped_image_df['series_description_x'].map(series_description_encoder)

# Encode levels
label_encoder = {level: idx for idx, level in enumerate(cropped_image_df['severity'].unique())}
cropped_image_df['severity_encoder'] = cropped_image_df['severity'].map(label_encoder)



In [14]:
condition_encoder  = {cond : idx  for idx , cond in enumerate(cropped_image_df['condition_leve'].unique())}
cropped_image_df['condition_encode'] = cropped_image_df['condition_leve'].map(condition_encoder)

In [15]:

reverse_condition_encoder = {idx: cond for cond, idx in condition_encoder.items()}

In [16]:
condition_encoder['Spinal Canal Stenosis_L1/L2']

0

In [17]:
reverse_condition_encoder[0]

'Spinal Canal Stenosis_L1/L2'

In [18]:
cropped_image_df['series_description_encode'].unique()

array([0, 1, 2])

In [19]:
cropped_image_df.drop(columns = ['image_file_path' ,'condition' , 'condition_leve' ,'series_description_x' ,'severity'], inplace = True )

In [20]:
cropped_image_df.head(1)

,study_id,series_id,instance_number,level,x,y,file_path,series_description_y,series_description_encode,severity_encoder,condition_encode
0,4003253,702807833,8,L1/L2,322.831858,227.964602,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Sagittal T2/STIR,0,0,0


In [121]:
train_size  = int(0.8 * len(cropped_image_df))
val_size  = int(0.2 * len(cropped_image_df))
# train_size = 100
# val_size = 32
train_df, val_df = train_test_split(cropped_image_df, test_size=val_size, train_size=train_size)
# Create data generators
train_generator = CustomSeverityDataGenerator(train_df, batch_size=32)
val_generator = CustomSeverityDataGenerator(val_df, batch_size=32)

In [22]:
batch_inputs, batch_targets = train_generator[0]

# Unpack inputs
cropped_images, batch_descriptions, conditions = batch_inputs
labels = batch_targets

# Print shapes
print("Cropped Images shape:", cropped_images.shape)
print("Batch Descriptions shape:", batch_descriptions.shape)
print("Conditions shape:", conditions.shape)
print("Labels shape:", labels.shape)


Cropped Images shape: (32, 224, 224, 3)
Batch Descriptions shape: (32,)
Conditions shape: (32,)
Labels shape: (32,)


# Severirty

In [ ]:
input_shape = (224, 224, 3)

def create_severity_model(input_shape):
    image_input = Input(shape=input_shape, name='image_input')
    series_description_input = Input(shape=(1,), name='series_description')
    condition_level_input = Input(shape=(1,), name='condition_level')
    
    base_model = EfficientNetB0(weights=None, include_top=False, input_tensor=image_input)
    base_model.load_weights("/kaggle/input/pretrained-model/efficientnetb0_notop.h5")
    
    x = base_model.output
    x = Flatten()(x)
    metadata_concatenated = concatenate([series_description_input, condition_level_input])
    concatenated = concatenate([x, metadata_concatenated])

    x = Dense(256, activation='relu')(concatenated)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(3, activation='softmax')(x)
    
    model = Model(inputs=[image_input, series_description_input, condition_level_input], outputs=output)
    return model

severity_model = create_severity_model(input_shape)


In [123]:
severity_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
severity_model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ image_input[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_2     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ normalization_2[… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_

 Total params: 20,148,006 (76.86 MB)

 Trainable params: 20,105,983 (76.70 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [124]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint('cropped_model_checkpoint.keras', save_best_only=True)


# Train the model
print("Starting model training...")
history = severity_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=2,
    callbacks=[early_stopping, checkpoint],
    verbose=1
)
print("Model training completed.")
model_path = "cropped_model.h5"
severity_model.save(model_path)
print(f"Model  is saved in {model_path}")


Starting model training...
Epoch 1/2


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1188/1216 ━━━━━━━━━━━━━━━━━━━━ 17s 640ms/step - accuracy: 0.8078 - loss: 0.5343

/tmp/ipykernel_34/1614249765.py:57: RuntimeWarning: invalid value encountered in divide
  cropped_image = cropped_image / np.max(cropped_image)


1216/1216 ━━━━━━━━━━━━━━━━━━━━ 1037s 788ms/step - accuracy: 0.8080 - loss: nan - val_accuracy: 0.0606 - val_loss: nan
Epoch 2/2
1216/1216 ━━━━━━━━━━━━━━━━━━━━ 804s 658ms/step - accuracy: 0.0635 - loss: nan - val_accuracy: 0.0606 - val_loss: nan
Model training completed.
Model  is saved in cropped_model.h5


In [29]:
severity_model.evaluate(val_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step - accuracy: 0.8125 - loss: 0.6295


[0.6295300722122192, 0.8125]

In [30]:

severity_path = '/kaggle/working/cropped_model.h5'
sevrity_model = tf.keras.models.load_model(severity_path)



In [ ]:
def severity_prediction(image_path, description, condition, input_shape=(224, 224)):
    
    meta_data = pydicom.dcmread(image_path)
    pixel_array = meta_data.pixel_array
    image_data = cv2.resize(pixel_array, input_shape)
    image_data = image_data / np.max(image_data)
    image_data = np.stack((image_data,) * 3, axis=-1)
    image_data = np.expand_dims(image_data, axis=0)
    
    description = np.expand_dims(np.array(description), axis=0)
    condition = np.expand_dims(np.array(condition), axis=0)
 
    prediction = sevrity_model.predict([image_data, description, condition])
    probabilities = tf.nn.softmax(prediction).numpy()
    normal_mild, moderate, severe = probabilities[0]
    
    print(f"normal/mild: {normal_mild}\nmoderate: {moderate}\nsevere: {severe}")
    return normal_mild, moderate, severe


In [32]:
severity_prediction('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/2828203845/10.dcm', 0 , 24,)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
normal/mild: 0.43473291397094727
moderate: 0.31528612971305847
severe: 0.24998103082180023


(0.4347329, 0.31528613, 0.24998103)

In [33]:
files_lists = []
series_list = []
series_descriptions = []
for root, _, files in os.walk('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images'):
        for file in files:
            if file.endswith(".dcm"):
                fixed_part = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/"
                
                image_path = os.path.join(root, file)
                series_and_beyond = image_path.split(fixed_part)[1]
                series_id = series_and_beyond.split("/")[0]
                if  series_id ==  '2828203845':
                    series_description = 'Sagittal T1'
                if series_id == '3481971518':
                    series_description = 'Axial T2'
                if series_id  == '3844393089':
                    series_description = 'Sagittal T2/STIR'
                    
                    
                files_lists.append(image_path)
                series_list.append(series_id) 
                series_descriptions.append(series_description)
                


In [34]:
train_image_df = pd.DataFrame({
    "file_path":files_lists,
    "series_id":series_list, 
    'series_description':series_descriptions
})

In [35]:
train_image_df.iloc[96]

file_path             /kaggle/input/rsna-2024-lumbar-spine-degenerat...
series_id                                                    2828203845
series_description                                          Sagittal T1
Name: 96, dtype: object

In [36]:
cropped_image_df.head(2)

,study_id,series_id,instance_number,level,x,y,file_path,series_description_y,series_description_encode,severity_encoder,condition_encode
0,4003253,702807833,8,L1/L2,322.831858,227.964602,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Sagittal T2/STIR,0,0,0
1,4003253,702807833,8,L2/L3,320.571429,295.714286,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Sagittal T2/STIR,0,0,1


In [37]:
series_encoder_test = {ser : idx for idx , ser in enumerate(train_image_df['series_description'].unique())}
train_image_df['series_encoder'] = train_image_df['series_description'].map(series_encoder_test)

In [38]:
reverse_series_encoder_test = {idx : ser for ser ,idx in series_encoder_test.items()}

In [39]:
reverse_series_encoder_test[0]

'Sagittal T2/STIR'

In [40]:
train_image_df['series_encoder'].unique()

array([0, 1, 2])

In [41]:
train_image_df.head(2)

,file_path,series_id,series_description,series_encoder
0,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,3844393089,Sagittal T2/STIR,0
1,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,3844393089,Sagittal T2/STIR,0


In [42]:

def cropped_image_data(image_data, description,condition,  mask, new_shape=(224 , 224)):
        xmin, ymin, xmax, ymax = get_bounding_box(mask)
        cropped_image = image_data[ymin:ymax + 1, xmin:xmax + 1]
        cropped_image = cv2.resize(cropped_image, new_shape)
        cropped_image = cropped_image / np.max(cropped_image)
        return cropped_image, description,condition 

def get_bounding_box(mask):
        rows = np.any(mask, axis=1)
        cols = np.any(mask, axis=0)
        y_min, y_max = np.where(rows)[0][[0, -1]]
        x_min, x_max = np.where(cols)[0][[0, -1]]
        return x_min, y_min, x_max, y_max

In [112]:
def overall_prediction(dataframe, segmentation_model, severity_model):
    row_ids = []
    normal_milds = []
    moderates = []
    severes = []
    
    for index, row in dataframe.iterrows():  # Use dataframe.iterrows() to iterate over rows
        file_path = row['file_path']
        series_desc = row['series_encoder']
        image_shape = (224 ,224)
        meta_data = pydicom.dcmread(file_path)
        pixel_array = meta_data.pixel_array
        image_data = cv2.resize(pixel_array, image_shape)
        image_data = image_data / np.max(image_data)
        image_data = np.stack((image_data,)*3, axis=-1)
        image_data_pred  = np.expand_dims(image_data, axis=0)
        description= np.expand_dims(np.array(series_desc), axis=0)
        mask_data, label = segmentation_model.predict([image_data_pred, description])
        label = np.argmax(label)
        label_encode  = reverse_label_encoder[label]
        if label_encode !='No condition':

            cropped_image, series_desc, condition = cropped_image_data(image_data, series_desc, label, mask_data)
            description= np.expand_dims(np.array(series_desc), axis=0)

            # Prepare inputs for prediction
            cropped_image = np.expand_dims(cropped_image, axis=0)
            condition = np.expand_dims(np.array(condition), axis=0)

            # Perform prediction with severity model
            prediction = severity_model.predict([cropped_image,description,condition])
            probabilities = tf.nn.softmax(prediction).numpy()
            normal_mild, moderate, severe = probabilities[0]

            # Generate row_id
            row_id = f"44036939_{label_encode}"

            # Append results to lists
            row_ids.append(row_id)
            normal_milds.append(normal_mild)
            moderates.append(moderate)
            severes.append(severe)

        # Create submission dataframe
        submission_df = pd.DataFrame({
            'row_id': row_ids,
            'normal_mild': normal_milds,
            "moderate":moderates , 
            'severe': severes
        })

        # Save submission dataframe to CSV
        submission_df.to_csv("submission.csv", index=False)

# Example usage
overall_prediction(train_image_df, segmentation_model, severity_model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━

In [113]:
submission_df  = pd.read_csv('submission.csv')

In [116]:
submission_df.head()

,row_id,normal_mild,moderate,severe
0,44036939_No condition,0.433833,0.316020,0.250146
1,44036939_No condition,0.434336,0.315562,0.250101
2,44036939_No condition,0.434853,0.315111,0.250036
3,44036939_No condition,0.435469,0.314559,0.249972
4,44036939_No condition,0.435667,0.314404,0.249929
